In [12]:
import torch
import numpy as np
from numpy import loadtxt
from PIL import Image
import os, glob

from top_view_represent import top_view_representation
from monodepth2_master import depth_estimation
from DeepLabV3Plus_Pytorch_master import segmentation_predict, network

################ image_path = image directory
image_path = "./data/image_02/data"
ext = "png"
if os.path.isfile(image_path):
    # Only testing on a single image
    paths = [image_path]
elif os.path.isdir(image_path):
    # Searching folder for images
    paths = glob.glob(os.path.join(image_path, '*.{}'.format(ext)))
else:
    raise Exception("Can not find args.image_path: {}".format(image_path))

output_directory = "./data/top_view_result"
os.makedirs(output_directory, exist_ok=True)
    
    
##### get NN models
segmentation_model = network.modeling.__dict__['deeplabv3plus_mobilenet'](num_classes=19)
segmentation_model.load_state_dict( torch.load("./DeepLabV3Plus_Pytorch_master/best_deeplabv3plus_mobilenet_cityscapes_os16.pth" , map_location=torch.device('cpu') )['model_state'])
    
    
for image_path in paths:
    
    ######## get depth numpy array
    depth_np = depth_estimation.depth_estimation_monodepth2(image_path)

    ######## get segmentation image
    seg_np = segmentation_predict.DeepLabV3Plus_segmentation(image_path, segmentation_model)

    ######## get top-view representation image
    # get intrinsic matrix
    K_path =  "data/image_02/cam.txt"
    pil_seg_img = top_view_representation(depth_np, seg_np, image_path, K_path)

    # save top-view image
    image_name = image_path.split('/')[-1]
    output_name = os.path.splitext(os.path.basename(image_path))[0]
    file_path = os.path.join(output_directory, "{}_depth.png".format(output_name))

    pil_seg_img.save(file_path)

    #pil_seg_img.show()